# crypto_correlation

In [109]:
starting_date = '2017-01-01T00:00:00Z'
ending_date = '2022-04-29T00:00:00Z'
pair = 'BTC/USDT'
tf = "1d"
exchange = 'Binance'

to_present = True
pair_id = pair.split('/')[0] + "-" + pair.split('/')[-1]
print("Pair : " + pair_id)

Pair : BTC-USDT


In [110]:
import ccxt
from datetime import datetime as dt
from freqtrade.data.converter import ohlcv_to_dataframe
import pandas as pd
import time

exchange_dict = {'Binance':ccxt.binance(),
                'Kucoin':ccxt.kucoin(),
                'Coinex':ccxt.coinex(),
                'Kraken':ccxt.kraken(),
                'Coinbase':ccxt.coinbase()}
ex = exchange_dict[exchange]

print(ex.iso8601(ex.milliseconds()), 'Loading markets')
#markets = ex.load_markets()
print(ex.iso8601(ex.milliseconds()), 'Markets loaded')

from_timestamp = ex.parse8601(starting_date)
if to_present:
    now = ex.milliseconds()
else:
    now = ex.parse8601(ending_date)
df_list = []
while from_timestamp < now:
    print('Fetching candles starting from', ex.iso8601(from_timestamp))
    try:
        ohlcvs = ex.fetch_ohlcv(pair, tf, from_timestamp)
    except:
        print("Retrying...")
        continue
    time.sleep(ex.rateLimit / 1000)
    if not len(ohlcvs):
        break
    from_timestamp = ohlcvs[-1][0] + ex.parse_timeframe(tf) * 1000
    df_list.append(ohlcv_to_dataframe(ohlcvs, tf, pair=pair, drop_incomplete=False))

2022-04-29T06:31:10.822Z Loading markets
2022-04-29T06:31:10.822Z Markets loaded
Fetching candles starting from 2017-01-01T00:00:00.000Z
Fetching candles starting from 2018-12-30T00:00:00.000Z
Fetching candles starting from 2020-05-13T00:00:00.000Z
Fetching candles starting from 2021-09-25T00:00:00.000Z


In [160]:
df = pd.concat(df_list, axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
print('Starting date :', df['date'].iloc[0])
print('Ending date :', df['date'].iloc[-1])
print('DF is created.')
print('DF size: ', df.shape[0], 'rows')
df.head()

df.columns = ["date", "open_"+pair, "high_"+pair, "low_"+pair, "close_"+pair, "volume_"+pair]
df["date"] = pd.to_datetime(df["date"],format='%Y-%m-%d%H:%M').dt.strftime('%Y-%m-%d %H:%M')
df.sort_values(by="date", inplace=True)
df.drop("volume_"+pair, inplace=True, axis=1)
df.set_index("date", inplace=True)
df.head()

Starting date : 2017-08-17 00:00:00+00:00
Ending date : 2022-04-29 00:00:00+00:00
DF is created.
DF size:  1717 rows


,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT
date,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37
2017-08-19 00:00,4108.37,4184.69,3850.00,4139.98
2017-08-20 00:00,4120.98,4211.08,4032.62,4086.29
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00


In [161]:
# Daily return:
import numpy as np
ret = df["close_"+pair] / df["open_"+pair] - 1
print(ret.describe())
df['ret_'+pair] = ret
ret = ret.values
df.head()

count    1717.000000
mean        0.002193
std         0.041707
min        -0.395053
25%        -0.016096
50%         0.001771
75%         0.019728
max         0.225832
dtype: float64


,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT,ret_BTC/USDT
date,,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08,0.005538
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37,-0.041238
2017-08-19 00:00,4108.37,4184.69,3850.00,4139.98,0.007694
2017-08-20 00:00,4120.98,4211.08,4032.62,4086.29,-0.008418
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00,-0.013057


In [162]:
n_dev = 1
big_ret = ( ret >= (ret.mean() + n_dev*ret.std()) )
small_ret = ( ret <= (ret.mean() - n_dev*ret.std()) )
med_ret = ( ((ret.mean() + n_dev*ret.std()) > ret) & (ret > (ret.mean() - n_dev*ret.std())))
print(f"Daily Returns clustered: {big_ret.sum()}big, {med_ret.sum()}medium, {small_ret.sum()}small.")

Daily Returns clustered: 192big, 1336medium, 189small.


In [163]:
# https://www.investing.com/currencies/eur-usd-historical-data
df_gold = pd.read_csv("gold.csv")
df_gold.columns = ["date", "close_"+"GOLD", "open_"+"GOLD", "high_"+"GOLD", "low_"+"GOLD", "dummy_"+"GOLD"]
df_gold["date"] = pd.to_datetime(df_gold["date"])
df_gold["date"] = pd.to_datetime(df_gold["date"],format='%Y-%m-%d%H:%M').dt.strftime('%Y-%m-%d %H:%M')
df_gold.drop("dummy_"+"GOLD", inplace=True, axis=1)
df_gold.sort_values(by="date", inplace=True)
df_gold.set_index("date", inplace=True)

# changing 1,940.1 to 1940.1:
df_gold["close_"+"GOLD"] = df_gold["close_"+"GOLD"].apply(lambda s:
                                                          float(s.split(",")[0] + s.split(",")[1]))
df_gold["open_"+"GOLD"] = df_gold["open_"+"GOLD"].apply(lambda s:
                                                          float(s.split(",")[0] + s.split(",")[1]))
ret_gold = df_gold["close_"+"GOLD"] / df_gold["open_"+"GOLD"] - 1
print(ret_gold.describe())
df_gold['ret_'+"GOLD"] = ret_gold
df_gold.head()

count    1586.000000
mean        0.000090
std         0.008371
min        -0.057262
25%        -0.004190
50%         0.000453
75%         0.004478
max         0.049543
dtype: float64


,close_GOLD,open_GOLD,high_GOLD,low_GOLD,ret_GOLD
date,,,,,
2016-03-28 00:00,1221.05,1214.51,"1,222.96","1,208.55",0.005385
2016-03-29 00:00,1242.55,1221.40,"1,243.65","1,215.65",0.017316
2016-03-30 00:00,1224.85,1241.30,"1,243.30","1,223.33",-0.013252
2016-03-31 00:00,1232.70,1226.06,"1,240.05","1,224.81",0.005416
2016-04-01 00:00,1222.49,1232.68,"1,235.36","1,208.84",-0.008267


In [164]:
df_oil = pd.read_csv("oil.csv")
df_oil.columns = ["date", "close_"+"OIL", "open_"+"OIL", "high_"+"OIL", "low_"+"OIL", "dummy1_"+"OIL", "dummy2_"+"OIL"]
df_oil["date"] = pd.to_datetime(df_oil["date"])
df_oil["date"] = pd.to_datetime(df_oil["date"],format='%Y-%m-%d%H:%M').dt.strftime('%Y-%m-%d %H:%M')
df_oil.drop("dummy1_"+"OIL", inplace=True, axis=1)
df_oil.drop("dummy2_"+"OIL", inplace=True, axis=1)
df_oil.sort_values(by="date", inplace=True)
df_oil.set_index("date", inplace=True)

ret_oil = df_oil["close_"+"OIL"] / df_oil["open_"+"OIL"] - 1
print(ret_oil.describe())
df_oil['ret_'+"OIL"] = ret_oil
df_oil.head()

count    1607.000000
mean       -0.002240
std         0.092761
min        -3.122391
25%        -0.009646
50%         0.001706
75%         0.012149
max         0.204604
dtype: float64


,close_OIL,open_OIL,high_OIL,low_OIL,ret_OIL
date,,,,,
2016-03-28 00:00,39.39,39.55,40.14,38.86,-0.004046
2016-03-29 00:00,38.28,39.32,39.48,37.91,-0.026450
2016-03-30 00:00,38.32,38.52,39.85,38.14,-0.005192
2016-03-31 00:00,38.34,38.30,39.04,37.57,0.001044
2016-04-01 00:00,36.79,38.14,38.56,36.63,-0.035396


In [165]:
# Dropping NAN days:
pd.concat([df, df_gold, df_oil], axis=1).dropna(axis=0)

,open_BTC/USDT,high_BTC/USDT,low_BTC/USDT,close_BTC/USDT,ret_BTC/USDT,close_GOLD,open_GOLD,high_GOLD,low_GOLD,ret_GOLD,close_OIL,open_OIL,high_OIL,low_OIL,ret_OIL
date,,,,,,,,,,,,,,,
2017-08-17 00:00,4261.48,4485.39,4200.74,4285.08,0.005538,1288.01,1283.61,"1,290.47","1,282.20",0.003428,47.09,46.80,47.19,46.46,0.006197
2017-08-18 00:00,4285.08,4371.52,3938.77,4108.37,-0.041238,1284.50,1288.20,"1,301.20","1,283.64",-0.002872,48.51,46.93,48.74,46.78,0.033667
2017-08-21 00:00,4069.13,4119.62,3911.79,4016.00,-0.013057,1291.22,1283.60,"1,293.58","1,280.60",0.005936,47.37,48.72,48.75,47.03,-0.027709
2017-08-22 00:00,4016.00,4104.82,3400.00,4040.00,0.005976,1284.72,1291.18,"1,292.69","1,282.11",-0.005003,47.64,47.45,48.03,47.20,0.004004
2017-08-23 00:00,4040.00,4265.80,4013.89,4114.01,0.018319,1290.62,1285.56,"1,292.03","1,283.09",0.003936,48.41,47.64,48.50,47.53,0.016163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-22 00:00,40480.01,40795.06,39177.00,39709.18,-0.019042,1929.73,1951.49,"1,955.93","1,926.52",-0.011150,102.07,104.07,104.22,101.06,-0.019218
2022-04-25 00:00,39450.12,40616.00,38200.00,40426.08,0.024739,1897.69,1931.09,"1,934.71","1,891.05",-0.017296,98.54,101.38,101.55,95.28,-0.028013
2022-04-26 00:00,40426.08,40797.31,37702.26,38112.65,-0.057226,1906.66,1899.48,"1,911.30","1,895.79",0.003780,101.70,98.64,102.78,97.06,0.031022


In [170]:
df_ret = pd.concat([df["ret_"+pair], df_gold["ret_"+"GOLD"], df_oil["ret_"+"OIL"]], axis=1).dropna(axis=0)
df_ret.columns = [pair, "GOLD", "OIL"]
df_ret

,BTC/USDT,GOLD,OIL
date,,,
2017-08-17 00:00,0.005538,0.003428,0.006197
2017-08-18 00:00,-0.041238,-0.002872,0.033667
2017-08-21 00:00,-0.013057,0.005936,-0.027709
2017-08-22 00:00,0.005976,-0.005003,0.004004
2017-08-23 00:00,0.018319,0.003936,0.016163
...,...,...,...
2022-04-22 00:00,-0.019042,-0.011150,-0.019218
2022-04-25 00:00,0.024739,-0.017296,-0.028013
2022-04-26 00:00,-0.057226,0.003780,0.031022


In [191]:
print(f"correlation of {pair} and GOLD:")
print(np.corrcoef(df_ret[pair], df_ret["GOLD"]), "\n")

shift = 1
print(f"correlation of {pair} and GOLD[D ({shift})]:")
print(np.corrcoef(df_ret[pair][:-shift], df_ret["GOLD"][shift:]), "\n")

shift = -2
print(f"correlation of {pair} and GOLD[D ({shift})]:")
print(np.corrcoef(df_ret[pair][-shift:], df_ret["GOLD"][:shift]), "\n")

correlation of BTC/USDT and GOLD:
[[1.         0.07756482]
 [0.07756482 1.        ]] 

correlation of BTC/USDT and GOLD[D (1)]:
[[1.         0.02546243]
 [0.02546243 1.        ]] 

correlation of BTC/USDT and GOLD[D (-2)]:
[[1.         0.01657874]
 [0.01657874 1.        ]] 



In [200]:
for shift in range(1, 20):
    print(f"correlation of {pair} and GOLD[D{shift}]:",
          round(np.corrcoef(df_ret[pair][:-shift], df_ret["GOLD"][shift:])[0, 1], 3))

correlation of BTC/USDT and GOLD[D1]: 0.025
correlation of BTC/USDT and GOLD[D2]: 0.035
correlation of BTC/USDT and GOLD[D3]: 0.011
correlation of BTC/USDT and GOLD[D4]: 0.034
correlation of BTC/USDT and GOLD[D5]: 0.038
correlation of BTC/USDT and GOLD[D6]: 0.006
correlation of BTC/USDT and GOLD[D7]: -0.059
correlation of BTC/USDT and GOLD[D8]: -0.056
correlation of BTC/USDT and GOLD[D9]: -0.025
correlation of BTC/USDT and GOLD[D10]: -0.033
correlation of BTC/USDT and GOLD[D11]: 0.008
correlation of BTC/USDT and GOLD[D12]: 0.027
correlation of BTC/USDT and GOLD[D13]: 0.018
correlation of BTC/USDT and GOLD[D14]: 0.007
correlation of BTC/USDT and GOLD[D15]: 0.005
correlation of BTC/USDT and GOLD[D16]: 0.057
correlation of BTC/USDT and GOLD[D17]: -0.05
correlation of BTC/USDT and GOLD[D18]: 0.016
correlation of BTC/USDT and GOLD[D19]: -0.003


In [201]:
for shift in range(1, 20):
    print(f"correlation of {pair} and GOLD[D-{shift}]:",
          round(np.corrcoef(df_ret[pair][shift:], df_ret["GOLD"][:-shift])[0, 1], 3))

correlation of BTC/USDT and GOLD[D-1]: -0.005
correlation of BTC/USDT and GOLD[D-2]: 0.017
correlation of BTC/USDT and GOLD[D-3]: -0.028
correlation of BTC/USDT and GOLD[D-4]: -0.06
correlation of BTC/USDT and GOLD[D-5]: -0.058
correlation of BTC/USDT and GOLD[D-6]: -0.011
correlation of BTC/USDT and GOLD[D-7]: 0.002
correlation of BTC/USDT and GOLD[D-8]: 0.003
correlation of BTC/USDT and GOLD[D-9]: 0.04
correlation of BTC/USDT and GOLD[D-10]: -0.012
correlation of BTC/USDT and GOLD[D-11]: 0.046
correlation of BTC/USDT and GOLD[D-12]: 0.039
correlation of BTC/USDT and GOLD[D-13]: 0.044
correlation of BTC/USDT and GOLD[D-14]: -0.006
correlation of BTC/USDT and GOLD[D-15]: -0.042
correlation of BTC/USDT and GOLD[D-16]: -0.02
correlation of BTC/USDT and GOLD[D-17]: 0.012
correlation of BTC/USDT and GOLD[D-18]: -0.003
correlation of BTC/USDT and GOLD[D-19]: -0.008


In [202]:
for shift in range(1, 20):
    print(f"correlation of {pair} and OIL[D{shift}]:",
          round(np.corrcoef(df_ret[pair][:-shift], df_ret["OIL"][shift:])[0, 1], 3))

correlation of BTC/USDT and OIL[D1]: 0.031
correlation of BTC/USDT and OIL[D2]: -0.017
correlation of BTC/USDT and OIL[D3]: 0.0
correlation of BTC/USDT and OIL[D4]: 0.041
correlation of BTC/USDT and OIL[D5]: -0.005
correlation of BTC/USDT and OIL[D6]: 0.028
correlation of BTC/USDT and OIL[D7]: -0.013
correlation of BTC/USDT and OIL[D8]: -0.0
correlation of BTC/USDT and OIL[D9]: -0.033
correlation of BTC/USDT and OIL[D10]: -0.009
correlation of BTC/USDT and OIL[D11]: -0.005
correlation of BTC/USDT and OIL[D12]: -0.01
correlation of BTC/USDT and OIL[D13]: -0.01
correlation of BTC/USDT and OIL[D14]: -0.049
correlation of BTC/USDT and OIL[D15]: 0.004
correlation of BTC/USDT and OIL[D16]: 0.006
correlation of BTC/USDT and OIL[D17]: 0.004
correlation of BTC/USDT and OIL[D18]: -0.012
correlation of BTC/USDT and OIL[D19]: -0.079


In [203]:
for shift in range(1, 20):
    print(f"correlation of {pair} and OIL[D-{shift}]:",
          round(np.corrcoef(df_ret[pair][shift:], df_ret["OIL"][:-shift])[0, 1], 3))

correlation of BTC/USDT and OIL[D-1]: -0.018
correlation of BTC/USDT and OIL[D-2]: -0.039
correlation of BTC/USDT and OIL[D-3]: -0.026
correlation of BTC/USDT and OIL[D-4]: -0.003
correlation of BTC/USDT and OIL[D-5]: 0.004
correlation of BTC/USDT and OIL[D-6]: -0.042
correlation of BTC/USDT and OIL[D-7]: -0.062
correlation of BTC/USDT and OIL[D-8]: -0.01
correlation of BTC/USDT and OIL[D-9]: -0.002
correlation of BTC/USDT and OIL[D-10]: 0.003
correlation of BTC/USDT and OIL[D-11]: -0.014
correlation of BTC/USDT and OIL[D-12]: -0.042
correlation of BTC/USDT and OIL[D-13]: -0.05
correlation of BTC/USDT and OIL[D-14]: 0.018
correlation of BTC/USDT and OIL[D-15]: -0.004
correlation of BTC/USDT and OIL[D-16]: -0.024
correlation of BTC/USDT and OIL[D-17]: -0.034
correlation of BTC/USDT and OIL[D-18]: -0.01
correlation of BTC/USDT and OIL[D-19]: 0.018


In [ ]:
# Developed by Reza Alavi(2022)